<a href="https://colab.research.google.com/github/Jungyoonlim/middlemarch/blob/main/Middlemarch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
print(os.getcwd())

/content


In [12]:
from google.colab import files

uploaded = files.upload()

Saving middlemarch.txt to middlemarch.txt


In [13]:
file_path = 'middlemarch.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    middlemarch_text = file.read()

print("Text loaded successfully.")


Text loaded successfully.
